In [70]:
## Import dependencies 

import pandas as pd


In [71]:
## Read CSV file into Pandas DataFrame from URL 
## The target URL is the ASX company directory, this data will be scraped and stored in a PostgreSQL database 
## Creating a web-scraping programme will enable the database to be updated automatically when new companies are added to the index 

# Create URL object, assigning the target URL 

URL = "https://asx.api.markitdigital.com/asx-research/1.0/companies/directory/file?access_token=83ff96335c2d45a094df02a206a39ff4"

# Create data object to store results, use pd.read_csv to return DataFrame 

data = pd.read_csv(URL)

# Print results 

data

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D,1414 DEGREES LIMITED,Capital Goods,12/09/2018,11507186
1,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,8433630
2,1AE,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,12656725
3,1AG,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,7662078
4,1CG,ONE CLICK GROUP LIMITED,Capital Goods,28/04/2017,6859033
...,...,...,...,...,...
2055,ZLD,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,18722806
2056,ZMI,ZINC OF IRELAND NL,Materials,18/09/2007,4476030
2057,ZMM,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,4352035
2058,ZNC,ZENITH MINERALS LIMITED,Materials,29/05/2007,36999993


In [72]:
## Import dependencies 
# yahooquery library will be used to access quotes 

from yahooquery import Ticker

In [73]:
# Add the .AX suffix to the ASX code, this is required to exectue the yahooquery search

data['ASX code'] = data['ASX code'].astype(str) + '.AX'  

# Print results 

data

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D.AX,1414 DEGREES LIMITED,Capital Goods,12/09/2018,11507186
1,1AD.AX,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,8433630
2,1AE.AX,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,12656725
3,1AG.AX,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,7662078
4,1CG.AX,ONE CLICK GROUP LIMITED,Capital Goods,28/04/2017,6859033
...,...,...,...,...,...
2055,ZLD.AX,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,18722806
2056,ZMI.AX,ZINC OF IRELAND NL,Materials,18/09/2007,4476030
2057,ZMM.AX,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,4352035
2058,ZNC.AX,ZENITH MINERALS LIMITED,Materials,29/05/2007,36999993


In [83]:
data['Market Cap'] = data['Market Cap'].astype(int)

top200 = data.nlargest(200, 'Market Cap')

top200

ValueError: invalid literal for int() with base 10: 'SUSPENDED'

In [81]:
tickers = Ticker(data['ASX code'], asynchronous=True)

# Default period = ytd, interval = 1d
df = tickers.history()
df.head()

KeyboardInterrupt: 